In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
import os
import glob
import plotly.express as px

## Load and process our calcium imaging data for embedding
- metric used: entire dF/F0 vector

In [2]:
# where to get data?
input = '/Volumes/TOSHIBA/IMAGING/GCaMP_data/single_neuron'

# get all the unique genotypes/conditions and exlcude hidden files
paths = [x for x in os.listdir(input) if 'results' not in x]
paths = [x for x in paths if x!='.DS_Store']

# for storing data
curves = []
labels = []

# loop over each genotype x condition
for path in paths:
    # find raw datafile for each trial
    for file in glob.glob("{}/{}/**/Values.csv".format(input,path), recursive = True):
        with open(file) as f:
                lines = (line for line in f if not line.startswith('#'))
                try:
                    data = np.loadtxt(lines, delimiter=',', skiprows=1)
                    data = np.transpose(data)
                    data = data[-1]

                    data = data[0:14]
                    # compute dF/F0
                    data = (data-np.mean(data[0:4]))/np.mean(data[0:4])
                    # truncate to only response
                    # data = data[5:9]

                    for i in range(len(data)):
                        curves.append(data[i])
                    labels.append(path)
                except:
                    continue

X = np.array(curves).reshape((len(labels),int(len(curves)/len(labels))))
y = np.array(labels)

## tSNE embedding
- 3 components show clearer than 2 results
- low perplexity level to maintain local structures as all data are quite similar especially in regard to shape

In [7]:
# We want to get TSNE embedding with certain dimensions
n_components = 3
tsne = TSNE(n_components, perplexity =10, n_iter = 5000)
tsne_result = tsne.fit_transform(X)


/usr/local/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:800: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:810: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [8]:
tsne_result_df = pd.DataFrame({'tsne_1': tsne_result[:,0], 'tsne_2': tsne_result[:,1], 'tsne_3': tsne_result[:,2], 'label': y, 
    'driver':np.array([yi.split('_')[0] for yi in y]), 'effector': np.array([yi.split('_')[-1] for yi in y])})

tsne_result_df['neurotransmitter'] = ""
# annotate tsne dataframe by the type of neurotransmitter released
for i in range(len(tsne_result_df)):
    if tsne_result_df['driver'][i] == 'SS00739':
        tsne_result_df['neurotransmitter'][i] = 'Glut'
    elif tsne_result_df['driver'][i] in ['R20B01','R61D08']:
        tsne_result_df['neurotransmitter'][i] = 'Ach'
    else:
        tsne_result_df['neurotransmitter'][i] = 'GABA'

/var/folders/nm/487x6rtj1wqfzh7k0vfyx2mr0000gp/T/ipykernel_8590/2356657973.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nm/487x6rtj1wqfzh7k0vfyx2mr0000gp/T/ipykernel_8590/2356657973.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nm/487x6rtj1wqfzh7k0vfyx2mr0000gp/T/ipykernel_8590/2356657973.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [9]:
# visualize and color-code by label, for different effectors independently
# subset the result dataframe
viz = tsne_result_df[tsne_result_df['effector']=='UAS-Ab1-42']

colors ={
    'R20B01_UAS-GCaMP7s':'#0385ff', 
    'R55C05_UAS-GCaMP7s':'#f5720f', 
    'SS_TJ001_G2_UAS-GCaMP7s':'#f5720f',
    'R61D08_UAS-GCaMP7s':'#1e8f33',
    'SS00674_UAS-GCaMP7s':'#f5720f',
    'SS00739_UAS-GCaMP7s':'#1100ff',
    'SS00863_UAS-GCaMP7s':'#ffd000',
    'SS00888_UAS-GCaMP7s':'#f2027a',
    'R20B01_UAS-GCaMP7s_UAS-Ab1-40':'#0385ff', 
    'R55C05_UAS-GCaMP7s_UAS-Ab1-40':'#f5720f',
    'SS_TJ001_G2_UAS-GCaMP7s_UAS-Ab1-40':'#f5720f', 
    'R61D08_UAS-GCaMP7s_UAS-Ab1-40':'#1e8f33',
    'SS00674_UAS-GCaMP7s_UAS-Ab1-40':'#f5720f',
    'SS00739_UAS-GCaMP7s_UAS-Ab1-40':'#1100ff',
    'SS00863_UAS-GCaMP7s_UAS-Ab1-40':'#ffd000',
    'SS00888_UAS-GCaMP7s_UAS-Ab1-40':'#f2027a',
    'R20B01_UAS-GCaMP7s_UAS-Ab1-42':'#0385ff', 
    'R55C05_UAS-GCaMP7s_UAS-Ab1-42':'#f5720f',
    'SS_TJ001_G2_UAS-GCaMP7s_UAS-Ab1-42':'#f5720f', 
    'R61D08_UAS-GCaMP7s_UAS-Ab1-42':'#1e8f33',
    'SS00674_UAS-GCaMP7s_UAS-Ab1-42':'#f5720f',
    'SS00739_UAS-GCaMP7s_UAS-Ab1-42':'#1100ff',
    'SS00863_UAS-GCaMP7s_UAS-Ab1-42':'#ffd000',
    'SS00888_UAS-GCaMP7s_UAS-Ab1-42':'#f2027a'}

# 3D visualization
fig = px.scatter_3d(tsne_result_df, x = 'tsne_1',y = 'tsne_2',z='tsne_3', color = 'label',color_discrete_map = colors)
fig.update_traces(marker_size = 3)

fig.update_layout(
    width=800, height=600,
    margin=dict(t=30, r=0, l=30, b=10)
)
camera = dict(
    eye=dict(x=2, y=2, z=0.1),
)

fig.update_layout(scene_camera=camera)
fig.show()


In [18]:
# assign colors
# visualize and color-code by driver
colors ={
    'R20B01':'#0385ff', 
    'R55C05':'#f5720f', 
    'R61D08':'#1e8f33',
    'SS00674':'#f5720f',
    'SS00739':'#1100ff',
    'SS00863':'#ffd000',
    'SS00888':'#f2027a'}
    
# 3D visualization
fig = px.scatter_3d(tsne_result_df, x = 'tsne_1',y = 'tsne_2',z='tsne_3', color = 'driver',color_discrete_map = colors)
fig.update_traces(marker_size = 3)
fig.show()

In [5]:
# 3D visualization
import plotly.express as px
fig = px.scatter_3d(tsne_result_df, x = 'tsne_1',y = 'tsne_2',z='tsne_3', color = 'neurotransmitter',color_discrete_map = {'Ach':'deepskyblue','GABA':'gold','Glut':'mediumorchid'})
fig.update_traces(marker_size = 3)
fig.show()

In [6]:
# 3D visualization
import plotly.express as px
fig = px.scatter_3d(tsne_result_df, x = 'tsne_1',y = 'tsne_2',z='tsne_3', color = 'effector',color_discrete_map = {'UAS-GCaMP7s':'black','UAS-Ab1-40':'dimgrey', 'UAS-Ab1-42':'silver'})
fig.update_traces(marker_size = 3)
fig.show()